In [14]:
import pandas as pd
from pandas.io import gbq

In [42]:
# Extract, Transform, and Load data warehouse onto Google Big Query
def et_location(file):
    df = pd.read_csv("./flat_files/"+file)
    df = df[df["Code"].isin(["KHM", "LAO", "MMR", "THA", "VNM"])]
    if df["Year"].isin([2021]).any():
        df = df[df["Year"] == 2021]
    else:
        df = df.sort_values("Year", ascending=False).drop_duplicates(["Code"])
    df = df.drop(columns = ["Year"])
    return df

def merge(df1, df2):
    merge_data = pd.merge(df1, df2)
    return merge_data

def load_gbq(df, name):
    df.to_gbq(destination_table="cis-4400-baruch-hung-tran.project_cis4400."+name, project_id="cis-4400-baruch-hung-tran", if_exists="replace")

def main():
    # Location Data
    pop_data = et_location("world_population.csv")
    pop_data.rename(columns={'Population (historical estimates and future projections)': 'population'}, inplace=True)
    unempl_data = et_location("unemployment_rate.csv")
    unempl_data.rename(columns={'Unemployment, total (% of total labor force) (modeled ILO estimate)': 'unemployment_rate'}, inplace=True)
    hos_data = et_location("hospitals_beds_counts.csv")
    hos_data.rename(columns={'Hospital beds (total number)':'hospital_beds'}, inplace=True)
    gdp_data = et_location("world_gdp.csv")
    gdp_data.rename(columns={'GDP':'gdp'}, inplace=True)
    
    merge_data_1 = merge(pop_data, unempl_data)
    merge_data_2 = merge(merge_data_1, hos_data)
    location_data = merge(merge_data_2, gdp_data)
    
    # Covid Data
    cov_data = pd.read_csv("./flat_files/indochina_covid_data_daily.csv")
    cov_data = cov_data.fillna(-1) #fill NaN values with -1
    cov_data["tests_units"] = cov_data["tests_units"].replace(-1, "no info") # tests_units columns with string values
    cov_data.rename(columns={'iso_code':'Code', "location":"Entity"}, inplace=True)
    
    # Fact Table (contains all information merged between Covid Data & Location Data)
    merge_data = merge(cov_data, location_data)

    # Load onto Big Query
    load_gbq(pop_data, "pop_data")
    load_gbq(unempl_data, "unempl_data")
    load_gbq(hos_data, "hos_data")
    load_gbq(gdp_data, "gdp_data")
    load_gbq(location_data, "location_data")
    load_gbq(cov_data, "cov_data")
    load_gbq(merge_data, "all_data")
    
#     print("Data Loaded Successfully")
#     return merge_data

if __name__ == "__main__":
    main()

1it [00:05,  5.76s/it]
1it [00:03,  3.26s/it]
1it [00:03,  3.05s/it]
1it [00:04,  4.36s/it]
1it [00:04,  4.03s/it]
1it [00:06,  6.25s/it]
1it [00:06,  6.01s/it]


Data Loaded Successfully


,Code,Entity,date,total_cases,new_cases,total_deaths,new_deaths,new_tests,total_tests,positive_rate,tests_per_case,tests_units,total_vaccinations,people_vaccinated,people_fully_vaccinated,new_vaccinations,population,unemployment_rate,hospital_beds,gdp
0,KHM,Cambodia,6/1/2021,30710,616,220,6,4686.0,1162111.0,0.102,9.8,tests performed,4671201.0,2648859.0,2022342.0,57639.0,16946446,0.31,12417,5.954185e+10
1,KHM,Cambodia,6/2/2021,31460,750,230,10,5517.0,1167628.0,0.102,9.8,tests performed,4723944.0,2665822.0,2058122.0,52743.0,16946446,0.31,12417,5.954185e+10
2,KHM,Cambodia,6/3/2021,32189,729,236,6,5785.0,1173413.0,0.104,9.6,tests performed,4782587.0,2688014.0,2094573.0,58643.0,16946446,0.31,12417,5.954185e+10
3,KHM,Cambodia,6/4/2021,33075,886,242,6,4654.0,1178067.0,0.109,9.2,tests performed,4839260.0,2710359.0,2128901.0,56673.0,16946446,0.31,12417,5.954185e+10
4,KHM,Cambodia,6/5/2021,33613,538,252,10,5394.0,1183461.0,0.137,7.3,tests performed,4894315.0,2735199.0,2159116.0,55055.0,16946446,0.31,12417,5.954185e+10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
845,VNM,Vietnam,11/13/2021,1018346,8467,23018,88,-1.0,-1.0,-1.000,-1.0,no info,98930571.0,64322087.0,34608484.0,1098813.0,98168829,2.27,238458,6.610000e+11
846,VNM,Vietnam,11/14/2021,1026522,8176,23082,64,-1.0,-1.0,-1.000,-1.0,no info,99751224.0,64467940.0,35283284.0,820653.0,98168829,2.27,238458,6.610000e+11
847,VNM,Vietnam,11/15/2021,1035138,8616,23183,101,-1.0,-1.0,-1.000,-1.0,no info,100862898.0,64767521.0,36095377.0,1111674.0,98168829,2.27,238458,6.610000e+11
848,VNM,Vietnam,11/16/2021,1045397,10259,23270,87,-1.0,-1.0,-1.000,-1.0,no info,102030576.0,65222953.0,36807623.0,1167678.0,98168829,2.27,238458,6.610000e+11
